# Add Erroneous Data

Adding erroneous data to shrunk file for testing.

In [ ]:
import sqlite3

import pandas as pd
from sqlalchemy import create_engine

In [ ]:
!cp testing_SQ00014613.sqlite testing_err_SQ00014613.sqlite

In [ ]:
# create connections for sqlite
# reference: https://nih.figshare.com/articles/dataset/Cell_Health_-_Cell_Painting_Single_Cell_Profiles/9995672
sqlite_err_conn = create_engine("sqlite:///testing_err_SQ00014613.sqlite").connect()

In [ ]:
sql_stmt = """
UPDATE cytoplasm
SET Cytoplasm_Correlation_Costes_AGP_DNA = 'nan'
WHERE rowid = (select min(rowid) from cytoplasm);
"""
sqlite_err_conn.execute(statement=sql_stmt)

In [ ]:
sql_stmt = """
select * from cytoplasm
"""
pd.read_sql(sql=sql_stmt, con=sqlite_err_conn)["Cytoplasm_Correlation_Costes_AGP_DNA"]

In [ ]:
# close connections
sqlite_err_conn.close()

In [ ]:
# show relative filesizes
!ls -lh ./ | grep sqlite